In [2]:
import numpy as np
import pandas as pd
mydataML=pd.read_csv("C:/Users/OCTAVIO/Documents/Academic/Python/Machine Learning 1/data (1).csv")
mydataML.head()

,mora,atraso,vivienda,edad,dias_lab,exp_sf,nivel_ahorro,ingreso,linea_sf,deuda_sf,score,zona,clasif_sbs,nivel_educ
0,0,235,FAMILIAR,30,3748,93.0,5,3500.0,NaN,0.00,214,Lima,4,UNIVERSITARIA
1,0,18,FAMILIAR,32,4598,9.0,12,900.0,1824.67,1933.75,175,La Libertad,1,TECNICA
2,0,0,FAMILIAR,26,5148,8.0,2,2400.0,2797.38,188.29,187,Lima,0,UNIVERSITARIA
3,0,0,FAMILIAR,36,5179,20.0,12,2700.0,NaN,0.00,187,Ancash,0,TECNICA
4,0,0,FAMILIAR,46,3960,NaN,1,3100.0,2000.00,11010.65,189,Lima,0,TECNICA


In [3]:
stats=mydataML.describe()
dum_vivienda=pd.get_dummies(mydataML.vivienda,prefix="dum")
dum_zona=pd.get_dummies(mydataML.zona,prefix="dum")
dum_niveleduc=pd.get_dummies(mydataML.nivel_educ,prefix="dum")
mydataML=pd.concat([mydataML,dum_niveleduc,dum_vivienda,dum_zona],axis=1)
np.max(mydataML.ingreso),np.min(mydataML.ingreso),np.median(mydataML.ingreso)


(30000.0, 123.7, 2400.0)

In [4]:
percentilemydat=np.nanpercentile(mydataML.ingreso,range(0,101))
np.sum(mydataML.isnull())

mora                    0
atraso                  0
vivienda                0
edad                    0
dias_lab                0
exp_sf               1830
nivel_ahorro            0
ingreso                 0
linea_sf             1127
deuda_sf              461
score                   0
zona                    0
clasif_sbs              0
nivel_educ              0
dum_SECUNDARIA          0
dum_SIN EDUCACION       0
dum_TECNICA             0
dum_UNIVERSITARIA       0
dum_ALQUILADA           0
dum_FAMILIAR            0
dum_PROPIA              0
dum_Amazonas            0
dum_Ancash              0
dum_Apurimac            0
dum_Arequipa            0
dum_Ayacucho            0
dum_Cajamarca           0
dum_Callao              0
dum_Cuzco               0
dum_Huancavelica        0
dum_Huanuco             0
dum_Ica                 0
dum_Junin               0
dum_La Libertad         0
dum_Lambayeque          0
dum_Lima                0
dum_Loreto              0
dum_Madre de Dios       0
dum_Moquegua

In [5]:
# Llenado de NaNs mediante asignación de promedios
group_exp_sf=pd.DataFrame(mydataML.groupby(["nivel_educ"])["exp_sf"].mean()) 
group_linea_sf=pd.DataFrame(mydataML.groupby(["zona"])["linea_sf"].mean()) 
group_deuda_sf=pd.DataFrame(mydataML.groupby(["vivienda"])["deuda_sf"].mean())
group_deuda_sf.loc["ALQUILADA","deuda_sf"]
group_deuda_sf.loc["FAMILIAR","deuda_sf"]
educ={}
zonasperu={}
viv={}

for j in group_exp_sf.index:
    educ[j]=pd.DataFrame(mydataML.loc[mydataML["nivel_educ"]==j,"exp_sf"])
    educ[j]["exp_sf"].fillna(np.mean(educ[j]["exp_sf"]),inplace=True)

group_linea_sf.drop(["Huancavelica"],inplace=True)

for zonas in group_linea_sf.index:
    zonasperu[zonas]=pd.DataFrame(mydataML.loc[mydataML["zona"]==zonas,"linea_sf"])
    zonasperu[zonas]["linea_sf"].fillna(np.mean(zonasperu[zonas]["linea_sf"]),inplace=True)

for tviv in group_deuda_sf.index:
    viv[tviv]=pd.DataFrame(mydataML.loc[mydataML["vivienda"]==tviv,"deuda_sf"])
    viv[tviv]["deuda_sf"].fillna(np.mean(viv[tviv]["deuda_sf"]),inplace=True)

np.sum(viv["FAMILIAR"].isnull()),
np.sum(viv["PROPIA"].isnull())

deuda_sf    0
dtype: int64

In [6]:
# Reemplazando en la data original

for n in group_exp_sf.index:
    mydataML.loc[mydataML["nivel_educ"]==n,"exp_sf"]=educ[n]["exp_sf"]

for m in group_linea_sf.index:
    mydataML.loc[mydataML["zona"]==m,"linea_sf"]=zonasperu[m]["linea_sf"]

for cv in group_deuda_sf.index:
    mydataML.loc[mydataML["vivienda"]==cv,"deuda_sf"]=viv[cv]["deuda_sf"]

np.sum(mydataML.isnull())
mydataML.linea_sf.fillna(np.mean(mydataML.linea_sf),inplace=True)

In [7]:
#Tratamiento de outliers LI->2  LS->98

percentilemydat=np.nanpercentile(mydataML.ingreso,range(0,101))

mydataML["ingreso"]=np.where(mydataML["ingreso"]<=percentilemydat[1],percentilemydat[1],
         np.where(mydataML["ingreso"]>=percentilemydat[98],percentilemydat[98],mydataML.ingreso))

X=mydataML.loc[:,['mora', 'atraso', 'edad', 'dias_lab', 'exp_sf',
       'nivel_ahorro','linea_sf', 'deuda_sf', 'score',
       'clasif_sbs', 'dum_SECUNDARIA', 'dum_SIN EDUCACION',
       'dum_TECNICA', 'dum_UNIVERSITARIA', 'dum_ALQUILADA', 'dum_FAMILIAR',
       'dum_PROPIA', 'dum_Amazonas', 'dum_Ancash', 'dum_Apurimac',
       'dum_Arequipa', 'dum_Ayacucho', 'dum_Cajamarca', 'dum_Callao',
       'dum_Cuzco', 'dum_Huancavelica', 'dum_Huanuco', 'dum_Ica', 'dum_Junin',
       'dum_La Libertad', 'dum_Lambayeque', 'dum_Lima', 'dum_Loreto',
       'dum_Madre de Dios', 'dum_Moquegua', 'dum_Pasco', 'dum_Piura',
       'dum_Puno', 'dum_San Martin', 'dum_Tacna', 'dum_Tumbes', 'dum_Ucayali']]

Y=pd.DataFrame(mydataML.loc[:,'ingreso'])

In [8]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest=train_test_split(X, Y, test_size=0.3, random_state=5)

##################
#Árbol de decisión
##################
from sklearn.tree import DecisionTreeRegressor

#Objeto
arb_reg=DecisionTreeRegressor(max_depth=20)

#Entrenamiento
arb_reg.fit(Xtrain,Ytrain.ingreso)

#Predicción
pr_arb_train=arb_reg.predict(Xtrain)
pr_arb_test=arb_reg.predict(Xtest)
arb_reg.score(Xtrain,Ytrain.ingreso),arb_reg.score(Xtest,Ytest.ingreso)

(0.9958870293151003, 0.3464905968286356)

In [9]:
#Calibración
[np.mean(Ytrain.ingreso),np.mean(arb_reg.predict(Xtrain))], [np.mean(Ytest.ingreso),np.mean(arb_reg.predict(Xtest))] 

([4262.3536604864785, 4262.353660486477],
 [4273.648746031742, 4497.3265961215475])

In [10]:
#Importancia de las variables tratadas
impor=pd.DataFrame(arb_reg.feature_importances_)
impor.index=Xtrain.columns
impor.columns=['importancia']
impor.sort_values('importancia',ascending=False,inplace=True)

#Nuevas variables
Xtrain.columns
Xtrain_2=Xtrain.loc[:,['mora', 'atraso', 'edad', 'dias_lab', 'exp_sf', 'nivel_ahorro',
       'linea_sf', 'deuda_sf', 'score', 'clasif_sbs', 'dum_SECUNDARIA',
       'dum_SIN EDUCACION', 'dum_TECNICA', 'dum_UNIVERSITARIA',
       'dum_ALQUILADA', 'dum_FAMILIAR', 'dum_PROPIA', 'dum_Amazonas']]

Xtest_2=Xtest.loc[:,['mora', 'atraso', 'edad', 'dias_lab', 'exp_sf', 'nivel_ahorro',
       'linea_sf', 'deuda_sf', 'score', 'clasif_sbs', 'dum_SECUNDARIA',
       'dum_SIN EDUCACION', 'dum_TECNICA', 'dum_UNIVERSITARIA',
       'dum_ALQUILADA', 'dum_FAMILIAR', 'dum_PROPIA', 'dum_Amazonas']]
impor.head()

,importancia
linea_sf,0.360363
exp_sf,0.119481
dias_lab,0.114430
deuda_sf,0.084680
score,0.082058


In [11]:
#Optimizando profundidad con las nuevas variables
from sklearn.model_selection import GridSearchCV # Librería
# Generando objetos
space=np.array(range(1,50)) # Valores a probar
param={'max_depth':space} # Objeto a optimizar
arbol_param=DecisionTreeRegressor() # Objeto (árbol de decisión)
# Aplicación del Algoritmo
arbol_grid=GridSearchCV(arbol_param,param,cv=6) #Objeto
arbol_grid.fit(Xtrain_2,Ytrain.ingreso) #entrenamiento
# Revisando parámetros óptimos
arbol_grid.best_params_

{'max_depth': 6}

In [12]:
# R cuadrados
arbol_grid.score(Xtrain_2,Ytrain.ingreso), arbol_grid.score(Xtest_2,Ytest.ingreso)

(0.5181037279212066, 0.3420784279707718)

In [13]:
#Reentrenando Decision Tree Regressor con la profundidad optimizada y las nuevas variables

arb_reg2=DecisionTreeRegressor(max_depth=6)
#Entrenamiento
arb_reg2.fit(Xtrain_2,Ytrain.ingreso)

#Predicción
pr_arb_train2=arb_reg2.predict(Xtrain_2)
pr_arb_test2=arb_reg2.predict(Xtest_2)

#Performance R cuadrado
arb_reg2.score(Xtrain_2,Ytrain.ingreso) , arb_reg2.score(Xtest_2,Ytest.ingreso) 

(0.5181037279212064, 0.34222218491714307)

In [14]:
#Calibración
[np.mean(Ytrain.ingreso),np.mean(arb_reg2.predict(Xtrain_2))] , [np.mean(Ytest.ingreso),np.mean(arb_reg2.predict(Xtest_2))] 

([4262.3536604864785, 4262.353660486478],
 [4273.648746031742, 4359.471558238525])

In [15]:
##########################
#Support Vector Regressor
##########################

from sklearn.svm import SVR
# Objeto
SVRe=SVR()
# Entrenando 
SVRe.fit(Xtrain_2,Ytrain.ingreso)

#Predicción 
prob=SVRe.predict(Xtrain_2)

# Perfomance R cuadrado
SVRe.score(Xtrain_2,Ytrain.ingreso), SVRe.score(Xtest_2,Ytest.ingreso) 

(-0.08470770970561481, -0.09031768834311049)

In [16]:
[np.mean(Ytrain.ingreso),np.mean(prob)], [np.mean(Ytest.ingreso),np.mean(SVRe.predict(Xtest_2))] 

([4262.3536604864785, 2414.103957163793],
 [4273.648746031742, 2423.08557046502])

In [17]:
##########################
# Random Forest Regressor
##########################

# Libreria
from sklearn.ensemble import RandomForestRegressor
# Objeto
rf_reg=RandomForestRegressor(max_depth=6,n_estimators=10,random_state=5)
# Entrenando
rf_reg.fit(Xtrain_2,Ytrain.ingreso)

# Predicción
rf_reg.predict(Xtrain_2)
rf_reg.predict(Xtest_2)

# Performance: R cuadrado 
rf_reg.score(Xtrain_2,Ytrain.ingreso) , rf_reg.score(Xtest_2,Ytest.ingreso) 

(0.5712243678352475, 0.4252807656784374)

In [18]:
[np.mean(Ytrain.ingreso),np.mean(rf_reg.predict(Xtrain_2))] , [np.mean(Ytest.ingreso),np.mean(rf_reg.predict(Xtest_2))] 

([4262.3536604864785, 4247.207314048219],
 [4273.648746031742, 4320.116084348399])

In [19]:
##################
# Regresión Lineal
##################

# Libreria
from sklearn.linear_model import LinearRegression
lineal=LinearRegression()
lineal.fit(Xtrain_2,Ytrain.ingreso)

#Performance
lineal.score(Xtrain_2,Ytrain.ingreso) , lineal.score(Xtest_2,Ytest.ingreso) 

(0.4089957680128373, 0.3856049361399404)

In [20]:
#Predicción
lineal.predict(Xtrain_2)
lineal.predict(Xtest_2)

#Calibración
[np.mean(Ytrain.ingreso),np.mean(lineal.predict(Xtrain_2))] , [np.mean(Ytest.ingreso),np.mean(lineal.predict(Xtest_2))] 

([4262.3536604864785, 4262.353660486478],
 [4273.648746031742, 4358.238501893564])

In [21]:
##############################
#ESTABILIDAD DE LOS ALGORITMOS 
##############################

#Nota: Solo los algoritmos de árboles de regresión cuentan con las nuevas variables y la profundidad optimizada

Xtrain_2.columns
Xtrain.columns
bst_rc=[]
bst_1={} #Diccionario para variables de Xtrain 1
bst_2={} #Diccionario para variables de Xtrain 2

for v in Xtrain.columns:
    bst_1[v]=[]

for bs in Xtrain_2.columns:
    bst_2[bs]=[]


##Support Vector Regressor

for i in range(200):
    sample=np.random.choice(Xtrain.index,size=4000) # Sub-muestra
    muestra_x=Xtrain_2.loc[sample,:] # Generando X
    muestra_y=Ytrain.loc[sample,'ingreso'] # Generando y
    mod=SVR() # Objeto de la regresión
    mod.fit(muestra_x,muestra_y) # Entrenando
    # Almacenando resultados
    bst_rc.append(mod.score(muestra_x,muestra_y))


# Comparando
[np.mean(bst_rc),SVRe.score(Xtrain_2,Ytrain.ingreso)]

[-0.09909617172063957, -0.08470770970561481]

In [22]:
##Decision Tree Regressor
for i in range(2000):
    sample=np.random.choice(Xtrain_2.index,size=4500) # Sub-muestra
    muestra_x=Xtrain_2.loc[sample,:] # Generando X
    muestra_y=Ytrain.loc[sample,"ingreso"] # Generando y
    mod=DecisionTreeRegressor(max_depth=6) # Objeto de la regresión
    mod.fit(muestra_x,muestra_y) # Entrenando
    # Almacenando resultados
    bst_rc.append(mod.score(muestra_x,muestra_y))


# Comparando
[np.mean(bst_rc),arb_reg2.score(Xtrain_2,Ytrain.ingreso)] 

[0.5119657745846321, 0.5181037279212064]

In [23]:
##Random Forest Regressor
for i in range(1000):
    sample=np.random.choice(Xtrain_2.index,size=4500) # Sub-muestra
    muestra_x=Xtrain_2.loc[sample,:] # Generando X
    muestra_y=Ytrain.loc[sample,"ingreso"] # Generando y
    mod=RandomForestRegressor(max_depth=6,n_estimators=10,random_state=5) # Objeto de la regresión
    mod.fit(muestra_x,muestra_y) # Entrenando
    # Almacenando resultados
    bst_rc.append(mod.score(muestra_x,muestra_y))
    
# Comparando
[np.mean(bst_rc),rf_reg.score(Xtrain_2,Ytrain.ingreso)]

[0.5482276546345755, 0.5712243678352475]

In [24]:
##Linear Regression
for i in range(3000):
    sample=np.random.choice(Xtrain_2.index,size=4500) # Sub-muestra
    muestra_x=Xtrain_2.loc[sample,:] # Generando X
    muestra_y=Ytrain.loc[sample,"ingreso"] # Generando y
    mod=LinearRegression()# Objeto de la regresión
    mod.fit(muestra_x,muestra_y) # Entrenando
    # Almacenando resultados
    bst_rc.append(mod.score(muestra_x,muestra_y))
    
    
# Comparando
[np.mean(bst_rc),lineal.score(Xtrain_2,Ytrain.ingreso)] 

[0.48237897919982947, 0.4089957680128373]

In [25]:
########################
#Análisis de calibración
########################
np.sum(mydataML.isnull())

#El algoritmo seleccionado es el de regresión lineal
import scipy.stats as st
alpha=0.05
st.norm.ppf(1-alpha/2)

#Calibración global
Ytrain['ing_inter']=pd.cut(Ytrain.ingreso,bins=np.percentile(Ytrain.ingreso,range(0,101,20)))
lin_predic_train=pd.DataFrame(lineal.predict(Xtrain_2))
lin_predic_train.columns=["predict"]
lin_predic_train["predic_cat"]=pd.cut(lin_predic_train.predict,bins=np.percentile(lin_predic_train.predict,range(0,101,20)))

lin_predic_test=pd.DataFrame(lineal.predict(Xtest_2))
lin_predic_test.columns=["predict"]

cal={'ingreso':[np.mean(Ytrain.ingreso)],'predic':[np.mean(lin_predic_train["predict"])]}

cal_global=pd.DataFrame(cal)
cal_global["LS"]=cal_global.predic+st.norm.ppf(1-alpha/2)*(np.var(lin_predic_train["predict"])/len(lin_predic_train["predict"]))**0.5
cal_global["LI"]=cal_global.predic-st.norm.ppf(1-alpha/2)*(np.var(lin_predic_train["predict"])/len(lin_predic_train["predict"]))**0.5
cal_global['Calibracion']=np.where((cal_global.ingreso>=cal_global.LI)&(cal_global.ingreso<=cal_global.LS),'Calibrado','Descalibrado')


lin_predic_train.predict.index=Ytrain.index
lin_predic_test.index=Ytest.index

In [26]:
# Uniendo predicciones para medir calibración
mydataML_unif=pd.concat([lin_predic_train.predict,lin_predic_test.predict])
mydataML_unif=pd.DataFrame(mydataML_unif)
mydataML_unif.columns=["predict"]
data_2=pd.concat([mydataML,mydataML_unif],axis=1)

##Calibración por nivel educativo
part1_niv_edu=data_2.groupby(['nivel_educ'])['ingreso'].agg(media_ingreso='mean',n='count')
part2_niv_edu=data_2.groupby(['nivel_educ'])['predict'].agg(media_predict='mean')
cal_niveduc=pd.concat([part2_niv_edu,part1_niv_edu],axis=1)

secu=pd.DataFrame(data_2.loc[data_2.nivel_educ=="SECUNDARIA","predict"])
sin_educ=pd.DataFrame(data_2.loc[data_2.nivel_educ=="SIN EDUCACION","predict"])
tecni=pd.DataFrame(data_2.loc[data_2.nivel_educ=="TECNICA","predict"])
uni=pd.DataFrame(data_2.loc[data_2.nivel_educ=="UNIVERSITARIA","predict"])
len(secu)
len(sin_educ)
len(data_2.loc[data_2.nivel_educ=="SECUNDARIA","predict"])

var_secu=pd.DataFrame(np.var(secu))
var_sin_educ=pd.DataFrame(np.var(sin_educ))
var_tecni=pd.DataFrame(np.var(tecni))
var_uni=pd.DataFrame(np.var(uni))

var_total=pd.concat([var_secu,var_sin_educ,var_tecni,var_uni])
var_total.index=cal_niveduc.index
cal_niveduc["var_tot"]=var_total

In [30]:
for u in cal_niveduc.index:
    cal_niveduc.loc[cal_niveduc.index==u,["LS"]]=(np.mean(data_2.loc[data_2.nivel_educ==u,"predict"]))+st.norm.ppf(1-alpha/2)*(cal_niveduc.loc[cal_niveduc.index==u,"var_tot"]/len(data_2.loc[data_2.nivel_educ==u,"predict"]))**0.5
    cal_niveduc.loc[cal_niveduc.index==u,["LI"]]=(np.mean(data_2.loc[data_2.nivel_educ==u,"predict"]))-st.norm.ppf(1-alpha/2)*(cal_niveduc.loc[cal_niveduc.index==u,"var_tot"]/len(data_2.loc[data_2.nivel_educ==u,"predict"]))**0.5

np.var(np.array([4,3,5,8,4,5]))
np.var(secu)
np.mean(secu)
cal_niveduc['Calibración']=np.where((cal_niveduc.media_ingreso>=cal_niveduc.LI)&(cal_niveduc.media_ingreso<=cal_niveduc.LS),'Calibrado','Descalibrado')
cal_niveduc['n_pct']=cal_niveduc.n/np.sum(cal_niveduc.n)
cal_niveduc['amplitud']=cal_niveduc.LS-cal_niveduc.LI
### Midiendo porcentaje de calibración
cal_niveduc.groupby(['Calibración'])['n_pct'].sum()

C:\Users\OCTAVIO\AppData\Roaming\Python\Python39\site-packages\numpy\core\fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


Calibración
Calibrado    1.0
Name: n_pct, dtype: float64

In [32]:
## Calibración por tipo de vivienda
part1_tip_viv=data_2.groupby(['vivienda'])['ingreso'].agg(media_ingreso='mean',n='count')
part2_tip_viv=data_2.groupby(['vivienda'])['predict'].agg(media_predict='mean')
cal_tip_viv=pd.concat([part2_tip_viv,part1_tip_viv],axis=1)

cal_tip_viv["LS"]=""
cal_tip_viv["LI"]=""

var_2={}
viv={}

for lñ in cal_tip_viv.index:
    var_2[lñ]=pd.DataFrame(np.var(pd.DataFrame(data_2.loc[data_2.vivienda==lñ,"predict"])))
    var_2[lñ].columns=["var_total"]

vari_total=[]

for t in var_2.keys():
    vari_total.append(var_2[t]["var_total"])

vari_total=pd.DataFrame(vari_total)
vari_total.index=cal_tip_viv.index
cal_tip_viv["var_total"]=vari_total.predict


for jp in cal_tip_viv.index:
    cal_tip_viv.loc[cal_tip_viv.index==jp,["LS"]]=(np.mean(data_2.loc[data_2.vivienda==jp,"predict"]))+st.norm.ppf(1-alpha/2)*(cal_tip_viv.loc[cal_tip_viv.index==jp,"var_total"]/len(data_2.loc[data_2.vivienda==jp,"predict"]))**0.5
    cal_tip_viv.loc[cal_tip_viv.index==jp,["LI"]]=(np.mean(data_2.loc[data_2.vivienda==jp,"predict"]))-st.norm.ppf(1-alpha/2)*(cal_tip_viv.loc[cal_tip_viv.index==jp,"var_total"]/len(data_2.loc[data_2.vivienda==jp,"predict"]))**0.5

cal_tip_viv['Calibración']=np.where((cal_tip_viv.media_ingreso>=cal_tip_viv.LI)&(cal_tip_viv.media_ingreso<=cal_tip_viv.LS),'Calibrado','Descalibrado')
cal_tip_viv['n_pct']=cal_tip_viv.n/np.sum(cal_tip_viv.n)
cal_tip_viv['amplitud']=cal_tip_viv.LS-cal_tip_viv.LI

### Midiendo porcentaje de calibración
cal_tip_viv.groupby(['Calibración'])['n_pct'].sum()

Calibración
Calibrado    1.0
Name: n_pct, dtype: float64

In [33]:
## Calibración por edad
part1_edad=data_2.groupby(['edad'])['ingreso'].agg(media_ingreso='mean',n='count')
part2_edad=data_2.groupby(['edad'])['predict'].agg(media_predict='mean')
cal_edad=pd.concat([part2_edad,part1_edad],axis=1)

cal_edad["LS"]=""
cal_edad["LI"]=""

var_3={}
edad={}

for lg in cal_edad.index:
    var_3[lg]=pd.DataFrame(np.var(pd.DataFrame(data_2.loc[data_2.edad==lg,"predict"])))
    var_3[lg].columns=["var_total"]

vari_total_2=[]

for tg in cal_edad.index:
    vari_total_2.append(var_3[tg]["var_total"])

vari_total_2=pd.DataFrame(vari_total_2)
vari_total_2.index=cal_edad.index
cal_edad["var_total"]=vari_total_2.predict

for pk in cal_edad.index:
    cal_edad.loc[cal_edad.index==pk,["LS"]]=(np.mean(data_2.loc[data_2.edad==pk,"predict"]))+st.norm.ppf(1-alpha/2)*(cal_edad.loc[cal_edad.index==pk,"var_total"]/len(data_2.loc[data_2.edad==pk,"predict"]))**0.5
    cal_edad.loc[cal_edad.index==pk,["LI"]]=(np.mean(data_2.loc[data_2.edad==pk,"predict"]))-st.norm.ppf(1-alpha/2)*(cal_edad.loc[cal_edad.index==pk,"var_total"]/len(data_2.loc[data_2.edad==pk,"predict"]))**0.5

cal_edad['Calibración']=np.where((cal_edad.media_ingreso>=cal_edad.LI)&(cal_edad.media_ingreso<=cal_edad.LS),'Calibrado','Descalibrado')
cal_edad['n_pct']=cal_edad.n/np.sum(cal_edad.n)
cal_edad['amplitud']=cal_edad.LS-cal_edad.LI

### Midiendo porcentaje de calibración
cal_edad.groupby(['Calibración'])['n_pct'].sum()

Calibración
Calibrado       0.539469
Descalibrado    0.460531
Name: n_pct, dtype: float64